the pinout diagram for connecting a 16x2 LCD to your Arduino without a potentiometer for contrast control:

### LCD to Arduino Pin Connections

| **LCD Pin** | **Function**       | **Arduino Pin (New Assignment)** |
|-------------|--------------------|----------------------------------|
| VSS (Pin 1) | Ground (GND)       | GND                              |
| VDD (Pin 2) | Power (+5V)        | 5V                               |
| V0 (Pin 3)  | Contrast Adjustment| GND (or use a 1kΩ resistor to GND)|
| RS (Pin 4)  | Register Select    | **Pin 2**                        |
| RW (Pin 5)  | Read/Write         | GND                              |
| E (Pin 6)   | Enable             | **Pin 3**                        |
| D4 (Pin 11) | Data Bit 4         | **Pin 4**                        |
| D5 (Pin 12) | Data Bit 5         | **Pin 5**                        |
| D6 (Pin 13) | Data Bit 6         | **Pin 6**                        |
| D7 (Pin 14) | Data Bit 7         | **Pin A0** (Analog Pin 0)        |
| A (Pin 15)  | LED Backlight (+)  | 5V (with a 220Ω resistor)        |
| K (Pin 16)  | LED Backlight (-)  | GND                              |


### Notes:

1. **V0 (Contrast Pin)**: Connect this pin to **GND** for a basic setup. If the contrast isn't satisfactory (i.e., the characters are either too dark or not visible), you can insert a fixed resistor (around 1kΩ) between `V0` and **GND**. This resistor helps to reduce the contrast slightly.

2. **A (Anode) Pin**: This pin is for the backlight. It should be connected to **5V** through a 220Ω resistor to limit the current and avoid damaging the backlight.

3. **K (Cathode) Pin**: Connect this pin to **GND**.


In [ ]:
#include <LiquidCrystal.h>

// Initialize the library with the new pin numbers
LiquidCrystal lcd(2, 3, 4, 5, 6, A0);

int leftPokePin = 9;           // Pin for left poke event
int leftWithPelletPin = 10;    // Pin for left poke with pellet event
int rightPokePin = 11;         // Pin for right poke event
int rightWithPelletPin = 12;   // Pin for right poke with pellet event
int pelletPin = 13;            // Pin for both pellet in well and pellet intake events

bool pelletInWell = false;  // State variable to track if pellet is in the well

void setup() {
  Serial.begin(115200);

  // Setup pins as outputs
  pinMode(leftPokePin, OUTPUT);         
  pinMode(leftWithPelletPin, OUTPUT);   
  pinMode(rightPokePin, OUTPUT);        
  pinMode(rightWithPelletPin, OUTPUT);  
  pinMode(pelletPin, OUTPUT);     

  // Ensure all pins start LOW
  digitalWrite(leftPokePin, LOW);
  digitalWrite(leftWithPelletPin, LOW);
  digitalWrite(rightPokePin, LOW);
  digitalWrite(rightWithPelletPin, LOW);
  digitalWrite(pelletPin, LOW);

  // Initialize the LCD and print a welcome message
  lcd.begin(16, 2);
  lcd.print("TDT Photometry");
  lcd.setCursor(0, 1); // Move to the second line
  lcd.print("System Ready");
  delay(2000); // Show the message for 2 seconds
  lcd.clear();
}

void loop() {
  if (Serial.available() > 0) {
    char eventType = Serial.read(); // Read the event type identifier
    Serial.print("Received event type: ");
    Serial.println(eventType); // Print the received event type for debugging

    lcd.clear(); // Clear the previous message
    lcd.print("Event: "); // Start with "Event: "

    switch (eventType) {
      case 'L': // Left poke event
        digitalWrite(leftPokePin, HIGH);
        delay(10);
        digitalWrite(leftPokePin, LOW);
        Serial.println("Left poke LED triggered");
        lcd.setCursor(0, 1); // Move to the second line
        lcd.print("Left Poke");
        break;

      case 'W': // Left with pellet event
        digitalWrite(leftWithPelletPin, HIGH);
        delay(10);
        digitalWrite(leftWithPelletPin, LOW);
        Serial.println("Left with pellet LED triggered");
        lcd.setCursor(0, 1);
        lcd.print("Left + Pellet");
        break;

      case 'R': // Right poke event
        digitalWrite(rightPokePin, HIGH);
        delay(10);
        digitalWrite(rightPokePin, LOW);
        Serial.println("Right poke LED triggered");
        lcd.setCursor(0, 1);
        lcd.print("Right Poke");
        break;

      case 'Q': // Right with pellet event
        digitalWrite(rightWithPelletPin, HIGH);
        delay(10);
        digitalWrite(rightWithPelletPin, LOW);
        Serial.println("Right with pellet LED triggered");
        lcd.setCursor(0, 1);
        lcd.print("Right + Pellet");
        break;

      case 'P': // Pellet in well or pellet intake event
        if (pelletInWell) {
          // Pellet is taken
          digitalWrite(pelletPin, LOW);   // Turn off the signal briefly
          delay(10);                      // Short delay
          pelletInWell = false;           // Update state
          Serial.println("Pellet taken: Signal turned OFF briefly");
          lcd.setCursor(0, 1);
          lcd.print("Pellet Taken");
        } else {
          // Pellet is in the well
          digitalWrite(pelletPin, HIGH);  // Keep the signal on
          pelletInWell = true;            // Update state
          Serial.println("Pellet in well: Signal turned ON");
          lcd.setCursor(0, 1);
          lcd.print("Pellet in Well");
        }
        break;

      default:
        Serial.println("Unknown event type received");
        lcd.setCursor(0, 1);
        lcd.print("Unknown Event");
        break;
    }
  }
}


In [1]:
import serial
import threading
import datetime

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    event_type = event_type.strip()  # Strip whitespace but keep the event type uppercase
    print(f"Normalized event type: '{event_type}'")

    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
        print("Sending 'R' for Right poke event")
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
        print("Sending 'L' for Left poke event")
    elif event_type in ["Pellet", "PelletInWell"]:
        arduino_ser.write(b'P')  # Pellet intake or pellet in well event
        print(f"Sending 'P' for {event_type} event")
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
        print("Sending 'W' for LeftWithPellet event")
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
        print("Sending 'Q' for RightWithPellet event")
    else:
        print(f"Unknown event type: '{event_type}'")
    print(f"Sent to Arduino: {event_type}")

def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Extract and print the event type
            event_type = data_list[8]  # "Event" field contains event type
            print(f"Extracted event type: '{event_type}'")
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()



Data from COM16: 10/1/2024 09:45:56,23.02,37.05,1.16.3,FR1,20,3.70,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.01
Extracted event type: 'Right'
Normalized event type: 'Right'
Sending 'R' for Right poke event
Sent to Arduino: Right
Data from COM16: 10/1/2024 09:45:58,23.02,37.01,1.16.3,FR1,20,3.70,NaN,1,Left,Left,1,1,0,0,NaN,NaN,0.11
Extracted event type: 'Left'
Normalized event type: 'Left'
Sending 'L' for Left poke event
Sent to Arduino: Left
Data from COM16: 10/1/2024 09:46:00,23.04,37.02,1.16.3,FR1,20,3.71,NaN,1,PelletInWell,Left,1,1,0,0,NaN,NaN,NaN
Extracted event type: 'PelletInWell'
Normalized event type: 'PelletInWell'
Sending 'P' for PelletInWell event
Sent to Arduino: PelletInWell
Data from COM16: 10/1/2024 09:46:06,23.05,36.98,1.16.3,FR1,20,3.70,2,1,Pellet,Left,1,1,1,0,5.59,NaN,NaN
Extracted event type: 'Pellet'
Normalized event type: 'Pellet'
Sending 'P' for Pellet event
Sent to Arduino: Pellet


Exception in thread Thread-5 (read_from_port):
Traceback (most recent call last):
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hta031\AppData\Local\Temp\ipykernel_12280\3011766281.py", line 42, in read_from_port
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))


Data from COM16: 
